In [22]:
from transformers import AutoTokenizer, VisionEncoderDecoderModel

tokenizer = AutoTokenizer.from_pretrained("kha-white/manga-ocr-base")
model = VisionEncoderDecoderModel.from_pretrained("kha-white/manga-ocr-base")



C:\Users\jinpaiduizhang\Anaconda3\envs\zhienv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToTensor
from tqdm import tqdm
import openpyxl

def jap_ocr(image_path):
    image = Image.open(image_path)
    resize = Resize((224, 224))
    image = resize(image)
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)
    
    generated_ids = model.generate(image_tensor)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [32]:
import os
from google.cloud import vision_v1
from google.oauth2 import service_account
from tqdm import tqdm
import openpyxl
import shutil

credentials_path = '../../credentials.json'
def google_ocr(image_path, credentials_path):
    # Initialize the client
    credentials = service_account.Credentials.from_service_account_file(credentials_path)
    client = vision_v1.ImageAnnotatorClient(credentials=credentials)

    # Read the image
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision_v1.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations# Extract and store the detected text in a list
    if texts:
        texts = texts[-1].description.split('\n')
    return texts


In [1]:
import pandas as pd
import glob

# Specify the path to your CSV files
path = "../../pilot data/classifypilot"  
all_files = glob.glob(path + "/*.csv")

# List to hold data from each CSV
li = []

# Loop through all files and read them into a dataframe, then append to the list
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concatenate all dataframes in the list into one dataframe
frame = pd.concat(li, axis=0, ignore_index=True)

# Optionally, you can save the merged dataframe to a new CSV file
frame.to_csv('merged.csv', index=False)

In [53]:
import pandas as pd

# image_folder_name = 'seg_letter'
image_folder_path = '../../pilot data/data/'# dir/dir/image
type_classifier = "merged.csv"

workbook = openpyxl.Workbook()
sheet = workbook.active
transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL Image to tensor
            # Add any additional transformations as needed
        ])
label_map = pd.read_csv(type_classifier, index_col=0).to_dict()['Class']
result = []
for img_dir_name in tqdm(os.listdir(image_folder_path)): 
    for filename in os.listdir(os.path.join(image_folder_path, img_dir_name)): 
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(image_folder_path, img_dir_name, filename)
            
            label = label_map.get(filename)
            if label == 'Kana':
                text = jap_ocr(image_path)
            elif label == 'Kanji':
                text = google_ocr(image_path,credentials_path)
                if text == []:
                    text = jap_ocr(image_path)
            else:
                text = "unknown label"
            result.append((img_dir_name, filename, text))

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [09:20<00:00, 70.12s/it]


In [54]:
from openpyxl import Workbook

def save_to_excel(results, excel_path):
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["group", "ocr", "filename"])
    for (img_dir_name, filename, text) in results:
        if text != []:
            combined_text = ' '.join(text) if isinstance(text, list) else text
            sheet.append((img_dir_name, filename, combined_text))
        else:
            sheet.append((img_dir_name, filename, ""))
    workbook.save(excel_path)
    workbook.close()
    df = pd.read_excel(excel_path)
    df.rename(columns={df.columns[0]: 'group', df.columns[1]: 'ocr', df.columns[2]: 'filename'}, inplace=True)
    df.to_excel(excel_path, index=False)
    return excel_path

excel_path = 'pilot_data_ocr1.xlsx'
save_to_excel(result, excel_path)